In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
print(os.listdir(data_dir))

The dataset includes 3670 images of flowers organized into five subdirectories: dandelion, roses, tulips, daisy, and sunflowers.

In [ ]:
roses = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[1]))

In [ ]:
daisy = list(data_dir.glob('daisy/*'))
PIL.Image.open(str(daisy[2]))

We resize the images in our dataset because they are of different sizes. We specify the image height and width to do this. We also specify the batch size, which is the number of images used by the model during each epoch.

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
# Create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(
    rotation_range=20, # Randomly rotate the image between -20 to 20 degrees
    width_shift_range=0.2, # Shift the image horizontally by a random factor between -0.2 to 0.2
    height_shift_range=0.2, # Shift the image vertically by a random factor between -0.2 to 0.2
    shear_range=0.2, # Apply shear transformation randomly with a factor between -0.2 to 0.2
    zoom_range=0.2, # Randomly zoom into the image by a factor between 0.8 to 1.2
    horizontal_flip=True, # Randomly flip the image horizontally
    fill_mode='nearest' # Fill the empty pixel positions using the nearest pixels
)

# Example usage: load images from a directory
training_generator = datagen.flow_from_directory(
    data_dir, # Path to the directory containing the training images
    target_size=(img_height, img_width),
    batch_size=batch_size, # Load the images in batches of 32
    class_mode='categorical' # Set the class mode to categorical (since we have multiple classes)
)
training_samples, __ = train_test_split(training_generator, test_size=0.2)

In [ ]:
val_generator = ImageDataGenerator()
validation_generator = val_generator.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode = 'categorical'
)
_, validation_samples = train_test_split(validation_generator, test_size=0.2)

Therefore, out of the total 3670 image files belonging to 5 classes, we are using 2936 image files for training and 734 image files for validation.

In [ ]:
class_names = training_generator.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 12))
for images, labels in training_generator.take(1):
  for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,activation='softmax')
])

In [ ]:
model2 = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=15
history = model.fit(
  training_samples,
  validation_data=validation_samples,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('model 1 Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('model 2 Training and Validation Loss')
plt.show()

In [ ]:
def predict_input_image(img):
  img_4d=img.reshape(-1,180,180,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(5)}

In [ ]:
import gradio as gr

Before creating Gradio’s user interface, we must specify the size of the picture that Gradio’s input component will store. As shown in the code below, we are also providing the number of labeled classes in the image dataset.

In [ ]:
image = gr.inputs.Image(shape=(180,180))
label = gr.outputs.Label(num_top_classes=5)

gr.Interface(fn=predict_input_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')